In [1]:
%%writefile decor_header.py
import random; from IPython.display import display,HTML
from IPython.core.magic import register_line_magic
@register_line_magic
def decor_header(params):
    params=params.split('|'); string=params[0]
    if len(params)==1: 
        font_size='22'; font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==2: 
        font_size=params[1]
        font_family='Wallpoet'; cmap='Sinebow'
    elif  len(params)==3: 
        font_size=params[1]; font_family=params[2]
        cmap='Sinebow'
    else: 
        font_size=params[1]; font_family=params[2]; cmap=params[3]
    height=max([int(font_size)*2.5,60]) 
    randi=str(random.uniform(0,9999999))
    html_str="""
<script src='https://d3js.org/d3.v6.min.js'></script>
<style>
@import 'https://fonts.googleapis.com/css?family="""+font_family+"""';
#colorized001 {
font-family:"""+font_family+"""; font-size:"""+font_size+""";}
#canvas001,#canvas002 {width:10%; vertical-align:middle;}
</style>
<text id='colorized001'><canvas id='canvas001'></canvas>
"""+string+"""
<canvas id='canvas002'></canvas></text><br/>
<script>
var tc=setInterval(function() {
    var now=(new Date().getTime()%5000)/5000;
    var now_slow=(new Date().getTime()%100000)/100000;
    var iddoc=document.getElementById('colorized001');
    iddoc.style.color=d3.interpolate"""+cmap+"""(now);
    var r=10,n=7;
    var c1=document.getElementById('canvas001'); 
    var context1=c1.getContext('2d');
    var c2=document.getElementById('canvas002'); 
    var context2=c2.getContext('2d');
    c1.style.background=d3.interpolate"""+cmap+"""(now_slow); 
    c2.style.background=d3.interpolate"""+cmap+"""(now_slow);
    context1.strokeStyle=d3.interpolate"""+cmap+"""(now);    
    context2.strokeStyle=d3.interpolate"""+cmap+"""(now);
    for (var i=1; i<n; i++) {
        context1.beginPath(); context2.beginPath();
        for (var j=0; j<6; j++) {
            context1.arc(60*j,r*(n+.5),i*r,0,2*Math.PI);
            context2.arc(60*j,r*(n+.5),i*r,0,2*Math.PI); };
        context1.stroke(); context2.stroke(); }; },1)
</script>"""
    file='d3header'+randi+'.html'
    with open(file,'w') as f:
         f.write(html_str); f.close()
    string="""<div id='html_string"""+randi+\
    """' style='width:100%;'><iframe src='"""+file+\
    """' height="""+str(height)+"""
    style='display:block; width:100%;'></iframe></div>"""
    display(HTML(string))

Writing decor_header.py


In [2]:
%run decor_header.py
%decor_header Modules, Helpful Functions, & Styling

In [3]:
!pip install --upgrade neural_structured_learning --user --quiet 

In [4]:
%%html
<style>
@import 'https://fonts.googleapis.com/css?family=Roboto';
span {text-shadow:3px 3px 3px #aaa;}  
div.output_area pre{font-family:Roboto; font-size:110%; color:#36ff36;}      
</style>

In [5]:
import warnings; warnings.filterwarnings('ignore')
import h5py,pylab as pl,pandas as pd
import numpy as np,tensorflow as tf
import neural_structured_learning as nsl
from sklearn.model_selection import train_test_split
file_path3='../input/classification-of-handwritten-letters/'
file_path4='../input/flower-color-images/'
model_weights='/checkpoints'

In [6]:
def prepro(x_train,y_train,x_test,y_test):
    n=int(len(x_test)/2)    
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    df=pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
                     [y_train.shape,y_valid.shape,y_test.shape]],
                    columns=['train','valid','test'],
                    index=['image arrays','label arrays'])
    display(df.style.set_table_styles(pd_style()))
    return [[x_train,x_valid,x_test],
            [y_train,y_valid,y_test]]
def pd_style():
    return [dict(selector='th',
                 props=[('font-size','12pt'),('min-width','150px')]),
            dict(selector='td',
                 props=[('padding','0em 0em'),('min-width','150px')]),
            dict(selector='tr:hover th:hover',
                 props=[('font-size','14pt'),('max-width','150px'),
                        ('text-shadow','3px 3px 3px #aaa')]),
            dict(selector='tr:hover td:hover',
                 props=[('font-size','12pt'),('max-width','150px'),
                        ('text-shadow','3px 3px 3px #aaa')])]
def cb(mw):
    early_stopping=tf.keras.callbacks.EarlyStopping(
        monitor='val_sparse_categorical_accuracy',
        patience=20,verbose=2,mode='max')
    checkpointer=tf.keras.callbacks.ModelCheckpoint(
        save_best_only=True,save_weights_only=True,
        monitor='val_sparse_categorical_accuracy',
        filepath=mw,verbose=2,mode='max')
    lr_reduction=tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_sparse_categorical_accuracy',
        verbose=2,patience=5,factor=.8,mode='max')
    return [checkpointer,early_stopping,lr_reduction]

In [7]:
%decor_header Data Loading & Preprocessing

In [8]:
(x_train1,y_train1),(x_test1,y_test1)=\
tf.keras.datasets.mnist.load_data()
[[x_train1,x_valid1,x_test1],
 [y_train1,y_valid1,y_test1]]=\
prepro(x_train1/255,y_train1.reshape(-1,1),
       x_test1/255,y_test1.reshape(-1,1))

11493376/11490434 [==============================] - 0s 0us/step


,train,valid,test
image arrays,"(60000, 28, 28)","(5000, 28, 28)","(5000, 28, 28)"
label arrays,"(60000, 1)","(5000, 1)","(5000, 1)"


In [9]:
(x_train2,y_train2),(x_test2,y_test2)=\
tf.keras.datasets.cifar10.load_data()
[[x_train2,x_valid2,x_test2],
 [y_train2,y_valid2,y_test2]]=\
prepro(x_train2/255,y_train2,x_test2/255,y_test2)

170500096/170498071 [==============================] - 3s 0us/step


,train,valid,test
image arrays,"(50000, 32, 32, 3)","(5000, 32, 32, 3)","(5000, 32, 32, 3)"
label arrays,"(50000, 1)","(5000, 1)","(5000, 1)"


In [10]:
f=h5py.File(file_path3+'LetterColorImages_123.h5','r') 
keys=list(f.keys()); print(keys)
images=np.array(f[keys[1]]).astype('float32')/255
labels=np.array(f[keys[2]]).astype('int32').reshape(-1,1)-1
x_train3,x_test3,y_train3,y_test3=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train3,x_valid3,x_test3],
 [y_train3,y_valid3,y_test3]]=\
prepro(x_train3,y_train3,x_test3,y_test3)

['backgrounds', 'images', 'labels']


,train,valid,test
image arrays,"(11352, 32, 32, 3)","(1419, 32, 32, 3)","(1419, 32, 32, 3)"
label arrays,"(11352, 1)","(1419, 1)","(1419, 1)"


In [11]:
f=h5py.File(file_path4+'FlowerColorImages.h5','r') 
keys=list(f.keys()); print(keys)
images=np.array(f[keys[0]]).astype('float32')/255
labels=np.array(f[keys[1]]).astype('int32').reshape(-1,1)
x_train4,x_test4,y_train4,y_test4=\
train_test_split(images,labels,test_size=.2,random_state=1)
del images,labels
[[x_train4,x_valid4,x_test4],
 [y_train4,y_valid4,y_test4]]=\
prepro(x_train4,y_train4,x_test4,y_test4)

['images', 'labels']


,train,valid,test
image arrays,"(168, 128, 128, 3)","(21, 128, 128, 3)","(21, 128, 128, 3)"
label arrays,"(168, 1)","(21, 1)","(21, 1)"


In [12]:
%decor_header Models with Adversarial Regularization
%decor_header MLP Base|18

In [13]:
batch_size=64; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size),name='input'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(n_class,activation=tf.nn.softmax)
])
adv_config=nsl.configs.make_adv_reg_config(
    multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras.AdversarialRegularization(
    base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [14]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1,'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1,'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 938 steps, validate for 78 steps
Epoch 1/7

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.94151, saving model to /checkpoints
938/938 - 8s - loss: 0.2664 - sparse_categorical_crossentropy: 0.2146 - sparse_categorical_accuracy: 0.9359 - scaled_adversarial_loss: 0.0517 - val_loss: 0.2316 - val_sparse_categorical_crossentropy: 0.1867 - val_sparse_categorical_accuracy: 0.9415 - val_scaled_adversarial_loss: 0.0450
Epoch 2/7

Epoch 00002: val_sparse_categorical_accuracy improved from 0.94151 to 0.95052, saving model to /checkpoints
938/938 - 6s - loss: 0.1119 - sparse_categorical_crossentropy: 0.0881 - sparse_categorical_accuracy: 0.9731 - scaled_adversarial_loss: 0.0239 - val_loss: 0.2019 - val_sparse_categorical_crossentropy: 0.1621 - val_sparse_categorical_accuracy: 0.9505 - val_scaled_adversarial_loss: 0.0398
Epoch 3/7

Epoch 00003: val_sparse_categorical_accuracy improved from 0.95052 to 0.95833, saving model to /checkpoints
938/938 - 5s - loss: 0.0727 -

In [15]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test1,'label':y_test1},verbose=0)))

{'loss': 0.07905462337408681,
 'sparse_categorical_crossentropy': 0.06229708,
 'sparse_categorical_accuracy': 0.9818,
 'scaled_adversarial_loss': 0.016380496}

In [16]:
batch_size=64; img_size=32; n_class=10; epochs=7
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [17]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 782 steps, validate for 78 steps
Epoch 1/7

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.37179, saving model to /checkpoints
782/782 - 12s - loss: 2.1236 - sparse_categorical_crossentropy: 1.7577 - sparse_categorical_accuracy: 0.3856 - scaled_adversarial_loss: 0.3659 - val_loss: 2.1770 - val_sparse_categorical_crossentropy: 1.8040 - val_sparse_categorical_accuracy: 0.3718 - val_scaled_adversarial_loss: 0.3730
Epoch 2/7

Epoch 00002: val_sparse_categorical_accuracy improved from 0.37179 to 0.42388, saving model to /checkpoints
782/782 - 8s - loss: 1.7692 - sparse_categorical_crossentropy: 1.4641 - sparse_categorical_accuracy: 0.4801 - scaled_adversarial_loss: 0.3051 - val_loss: 1.9893 - val_sparse_categorical_crossentropy: 1.6465 - val_sparse_categorical_accuracy: 0.4239 - val_scaled_adversarial_loss: 0.3428
Epoch 3/7

Epoch 00003: val_sparse_categorical_accuracy improved from 0.42388 to 0.45433, saving model to /checkpoints
782/782 - 8s - loss: 1.6315 

In [18]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test2,'label':y_test2},verbose=0)))

{'loss': 1.983568046951294,
 'sparse_categorical_crossentropy': 1.6397594,
 'sparse_categorical_accuracy': 0.433,
 'scaled_adversarial_loss': 0.3443603}

In [19]:
batch_size=128; img_size=32; n_class=33; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(33,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [20]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 89 steps, validate for 11 steps
Epoch 1/100

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.04901, saving model to /checkpoints
89/89 - 4s - loss: 3.3993 - sparse_categorical_crossentropy: 2.7746 - sparse_categorical_accuracy: 0.2230 - scaled_adversarial_loss: 0.6247 - val_loss: 4.5954 - val_sparse_categorical_crossentropy: 3.8157 - val_sparse_categorical_accuracy: 0.0490 - val_scaled_adversarial_loss: 0.7796
Epoch 2/100

Epoch 00002: val_sparse_categorical_accuracy improved from 0.04901 to 0.05043, saving model to /checkpoints
89/89 - 2s - loss: 2.3750 - sparse_categorical_crossentropy: 1.9140 - sparse_categorical_accuracy: 0.4349 - scaled_adversarial_loss: 0.4610 - val_loss: 8.1477 - val_sparse_categorical_crossentropy: 6.7483 - val_sparse_categorical_accuracy: 0.0504 - val_scaled_adversarial_loss: 1.3995
Epoch 3/100

Epoch 00003: val_sparse_categorical_accuracy did not improve from 0.05043
89/89 - 1s - loss: 1.9647 - sparse_categorical_crossentropy: 1

In [21]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test3,'label':y_test3},verbose=0)))

{'loss': 2.5792446825351405,
 'sparse_categorical_crossentropy': 1.9618881,
 'sparse_categorical_accuracy': 0.7040169,
 'scaled_adversarial_loss': 0.6380761}

In [22]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.models.Sequential([
        tf.keras.Input((img_size,img_size,3),name='input'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),    
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.BatchNormalization(),   
        tf.keras.layers.Dense(1024,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [23]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 11 steps, validate for 1 steps
Epoch 1/100

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.18750, saving model to /checkpoints
11/11 - 3s - loss: 3.5012 - sparse_categorical_crossentropy: 2.9029 - sparse_categorical_accuracy: 0.2321 - scaled_adversarial_loss: 0.5969 - val_loss: 9.4948 - val_sparse_categorical_crossentropy: 7.9093 - val_sparse_categorical_accuracy: 0.1875 - val_scaled_adversarial_loss: 1.5855
Epoch 2/100

Epoch 00002: val_sparse_categorical_accuracy did not improve from 0.18750
11/11 - 0s - loss: 1.9428 - sparse_categorical_crossentropy: 1.6109 - sparse_categorical_accuracy: 0.5774 - scaled_adversarial_loss: 0.3273 - val_loss: 10.0695 - val_sparse_categorical_crossentropy: 8.3880 - val_sparse_categorical_accuracy: 0.0000e+00 - val_scaled_adversarial_loss: 1.6815
Epoch 3/100

Epoch 00003: val_sparse_categorical_accuracy did not improve from 0.18750
11/11 - 0s - loss: 0.4032 - sparse_categorical_crossentropy: 0.3333 - sparse_categorical_acc

In [24]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,adv_model.evaluate(
    {'input':x_test4,'label':y_test4},verbose=0)))

{'loss': 2.478311777114868,
 'sparse_categorical_crossentropy': 2.0618477,
 'sparse_categorical_accuracy': 0.3809524,
 'scaled_adversarial_loss': 0.41646415}

In [25]:
%decor_header CNN Base|18

In [26]:
batch_size=128; img_size=28; n_class=10; epochs=7
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,1),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [27]:
train1=tf.data.Dataset.from_tensor_slices(
    {'input':x_train1.reshape(x_train1.shape[0],28,28,1),
     'label':y_train1}).batch(batch_size)
valid1=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid1.reshape(x_valid1.shape[0],28,28,1),
     'label':y_valid1}).batch(batch_size)
valid_steps=x_valid1.shape[0]//batch_size
adv_model.fit(train1,validation_data=valid1,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 469 steps, validate for 39 steps
Epoch 1/7

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.95793, saving model to /checkpoints
469/469 - 11s - loss: 0.4502 - sparse_categorical_crossentropy: 0.3733 - sparse_categorical_accuracy: 0.8783 - scaled_adversarial_loss: 0.0769 - val_loss: 0.1581 - val_sparse_categorical_crossentropy: 0.1293 - val_sparse_categorical_accuracy: 0.9579 - val_scaled_adversarial_loss: 0.0289
Epoch 2/7

Epoch 00002: val_sparse_categorical_accuracy improved from 0.95793 to 0.98277, saving model to /checkpoints
469/469 - 6s - loss: 0.1096 - sparse_categorical_crossentropy: 0.0895 - sparse_categorical_accuracy: 0.9728 - scaled_adversarial_loss: 0.0201 - val_loss: 0.0669 - val_sparse_categorical_crossentropy: 0.0542 - val_sparse_categorical_accuracy: 0.9828 - val_scaled_adversarial_loss: 0.0127
Epoch 3/7

Epoch 00003: val_sparse_categorical_accuracy did not improve from 0.98277
469/469 - 6s - loss: 0.0819 - sparse_categorical_crossentropy:

In [28]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test1.reshape(x_test1.shape[0],28,28,1),
              'label':y_test1},verbose=0)))

{'loss': 0.017785434613749384,
 'sparse_categorical_crossentropy': 0.014155323,
 'sparse_categorical_accuracy': 0.9954,
 'scaled_adversarial_loss': 0.0035471304}

In [29]:
batch_size=64; img_size=32; n_class=10; epochs=20
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(96,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [30]:
train2=tf.data.Dataset.from_tensor_slices(
    {'input':x_train2,'label':y_train2}).batch(batch_size)
valid2=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid2,'label':y_valid2}).batch(batch_size)
valid_steps=x_valid2.shape[0]//batch_size
adv_model.fit(train2,validation_data=valid2,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 782 steps, validate for 78 steps
Epoch 1/20

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.49159, saving model to /checkpoints
782/782 - 10s - loss: 1.9773 - sparse_categorical_crossentropy: 1.6428 - sparse_categorical_accuracy: 0.3952 - scaled_adversarial_loss: 0.3345 - val_loss: 1.6988 - val_sparse_categorical_crossentropy: 1.4053 - val_sparse_categorical_accuracy: 0.4916 - val_scaled_adversarial_loss: 0.2934
Epoch 2/20

Epoch 00002: val_sparse_categorical_accuracy improved from 0.49159 to 0.57031, saving model to /checkpoints
782/782 - 8s - loss: 1.5867 - sparse_categorical_crossentropy: 1.3140 - sparse_categorical_accuracy: 0.5272 - scaled_adversarial_loss: 0.2727 - val_loss: 1.4634 - val_sparse_categorical_crossentropy: 1.2057 - val_sparse_categorical_accuracy: 0.5703 - val_scaled_adversarial_loss: 0.2577
Epoch 3/20

Epoch 00003: val_sparse_categorical_accuracy improved from 0.57031 to 0.60938, saving model to /checkpoints
782/782 - 8s - loss: 1.44

In [31]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test2,'label':y_test2},verbose=0)))

{'loss': 1.0591906485557556,
 'sparse_categorical_crossentropy': 0.8531081,
 'sparse_categorical_accuracy': 0.703,
 'scaled_adversarial_loss': 0.20517391}

In [32]:
batch_size=128; img_size=32; n_class=33; epochs=150
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(33,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [33]:
train3=tf.data.Dataset.from_tensor_slices(
    {'input':x_train3,'label':y_train3}).batch(batch_size)
valid3=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid3,'label':y_valid3}).batch(batch_size)
valid_steps=x_valid3.shape[0]//batch_size
adv_model.fit(train3,validation_data=valid3,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 89 steps, validate for 11 steps
Epoch 1/150

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.05753, saving model to /checkpoints
89/89 - 3s - loss: 4.1903 - sparse_categorical_crossentropy: 3.4918 - sparse_categorical_accuracy: 0.0342 - scaled_adversarial_loss: 0.6985 - val_loss: 4.1750 - val_sparse_categorical_crossentropy: 3.4788 - val_sparse_categorical_accuracy: 0.0575 - val_scaled_adversarial_loss: 0.6962
Epoch 2/150

Epoch 00002: val_sparse_categorical_accuracy did not improve from 0.05753
89/89 - 1s - loss: 4.1033 - sparse_categorical_crossentropy: 3.4181 - sparse_categorical_accuracy: 0.0558 - scaled_adversarial_loss: 0.6852 - val_loss: 4.0535 - val_sparse_categorical_crossentropy: 3.3757 - val_sparse_categorical_accuracy: 0.0554 - val_scaled_adversarial_loss: 0.6778
Epoch 3/150

Epoch 00003: val_sparse_categorical_accuracy improved from 0.05753 to 0.06250, saving model to /checkpoints
89/89 - 1s - loss: 4.0141 - sparse_categorical_crossentropy: 3

In [34]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test3,'label':y_test3},verbose=0)))

{'loss': 0.430931953979765,
 'sparse_categorical_crossentropy': 0.31541225,
 'sparse_categorical_accuracy': 0.91120505,
 'scaled_adversarial_loss': 0.11484398}

In [35]:
batch_size=16; img_size=128; n_class=10; epochs=100
base_model=tf.keras.Sequential([
    tf.keras.Input((img_size,img_size,3),name='input'),
    tf.keras.layers.Conv2D(32,(5,5),padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Conv2D(196,(5,5)),
    tf.keras.layers.Activation('relu'),    
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.GlobalMaxPooling2D(),    
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(.25),
    tf.keras.layers.Dense(10,activation='softmax')
])
adv_config=nsl.configs\
.make_adv_reg_config(multiplier=.2,adv_step_size=.05)
adv_model=nsl.keras\
.AdversarialRegularization(base_model,adv_config=adv_config)
adv_model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')

In [36]:
train4=tf.data.Dataset.from_tensor_slices(
    {'input':x_train4,'label':y_train4}).batch(batch_size)
valid4=tf.data.Dataset.from_tensor_slices(
    {'input':x_valid4,'label':y_valid4}).batch(batch_size)
valid_steps=x_valid4.shape[0]//batch_size
adv_model.fit(train4,validation_data=valid4,verbose=2,
              validation_steps=valid_steps,
              epochs=epochs,callbacks=cb(model_weights));

Train for 11 steps, validate for 1 steps
Epoch 1/100

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.12500, saving model to /checkpoints
11/11 - 2s - loss: 2.7908 - sparse_categorical_crossentropy: 2.3284 - sparse_categorical_accuracy: 0.1250 - scaled_adversarial_loss: 0.4624 - val_loss: 2.7447 - val_sparse_categorical_crossentropy: 2.2866 - val_sparse_categorical_accuracy: 0.1250 - val_scaled_adversarial_loss: 0.4581
Epoch 2/100

Epoch 00002: val_sparse_categorical_accuracy improved from 0.12500 to 0.25000, saving model to /checkpoints
11/11 - 0s - loss: 2.5336 - sparse_categorical_crossentropy: 2.1125 - sparse_categorical_accuracy: 0.2560 - scaled_adversarial_loss: 0.4214 - val_loss: 2.5391 - val_sparse_categorical_crossentropy: 2.1150 - val_sparse_categorical_accuracy: 0.2500 - val_scaled_adversarial_loss: 0.4241
Epoch 3/100

Epoch 00003: val_sparse_categorical_accuracy did not improve from 0.25000
11/11 - 0s - loss: 2.1353 - sparse_categorical_crossentropy: 1.

In [37]:
adv_model.load_weights(model_weights)
dict(zip(adv_model.metrics_names,
         adv_model.evaluate(
             {'input':x_test4,'label':y_test4},verbose=0)))

{'loss': 0.8796605467796326,
 'sparse_categorical_crossentropy': 0.7273286,
 'sparse_categorical_accuracy': 0.8095238,
 'scaled_adversarial_loss': 0.15233193}